# Tutorial: Describe Neuroscience Dataset using MINDS

## Initialize and configure

In [ ]:
!pip install nexusforge==0.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 720 kB 5.0 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 45 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 41.9 MB/s 
     |████████████████████████████████| 482 kB 45.0 MB/s 
     |████████████████████████████████| 70 kB 7.9 MB/s 
     |████████████████████████████████| 1.2 MB 40.7 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 385 kB 50.0 MB/s 
     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 41 kB 611 kB/s 
     |████████████████████████████████| 144 kB 54.3 MB/s 
     |████████████████████████████████| 271 kB 49.3 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 99 kB 7.

In [ ]:
!pip install allensdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 118 kB 43.1 MB/s 
     |████████████████████████████████| 48.4 MB 30 kB/s 
     |████████████████████████████████| 3.0 MB 35.1 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 130 kB 16.0 MB/s 
     |████████████████████████████████| 185 kB 22.4 MB/s 
     |████████████████████████████████| 130 kB 9.3 MB/s 
     |████████████████████████████████| 1.3 MB 32.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 668 kB 49.4 MB/s 
     |████████████████████████████████| 3.4 MB 37.3 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 4.3 MB 36.4 MB/s 
     |██████████████████████

In [ ]:
!pip install neurom[plotly]==3.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 474 kB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.9 MB 43.4 MB/s 
     |████████████████████████████████| 281 kB 45.9 MB/s 
  Created wheel for neurom: filename=neurom-3.0.1-py3-none-any.whl size=100039 sha256=555e1106d8d1bab114c051e5937e8c55ccc3fd65421531b78126fc39c6675369
  Stored in directory: /root/.cache/pip/wheels/d3/47/0c/ad39d56ccedfb07bed19bde0dc51f2beafa786033117a036f3
Successfully built neurom
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
!pip install --upgrade nest-asyncio==1.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Get an authentication token

The [Nexus sandbox application](https://sandbox.bluebrainnexus.io) can be used to get a token:

- Step 1: From the [web page](https://sandbox.bluebrainnexus.io), click on the login button in the top right corner and follow the instructions on screen.

- Step 2: You will then see a `Copy token` button in the top right corner. Click on it to copy the token to the clipboard.


Once a token is obtained, proceed to paste it as the value of the `TOKEN` variable below.

__Important__: A Nexus token is valid for 8 hours, if your working session is open for more than 8 hours, you may need to refresh the value of the token and reintialize the forge client in the _'Configure a forge client to store, manage and access datasets'_ section below. 

In [ ]:
import getpass

In [ ]:
TOKEN = getpass.getpass()

··········


### Configure a forge client to store, manage and access datasets

In [ ]:
import uuid
import base64
import requests
import json
from pathlib import Path

from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.mappings import DictionaryMapping

from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import CellTypesCache

In [ ]:
r = requests.get('https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/rdfmodel/jsonldcontext.json')
dirpath = './rdfmodel'
Path(dirpath).mkdir(parents=True, exist_ok=True)
with open(f'{dirpath}/jsonldcontext.json', 'w') as outfile:
    json.dump(r.json(), outfile)

In [ ]:
ORG = "github-users"
PROJECT = "cdjcodes"  # Provide here the automatically created project name created when you logged into the Nexus sandbox instance.

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/forge.yml",
                            bucket=f"{ORG}/{PROJECT}",
                            endpoint="https://sandbox.bluebrainnexus.io/v1",
                            token=TOKEN)

## Download datasets from Allen Cell Types Database

### Download mouse neuron morphology from the Allen Cell Types Database

We will be downloading mouse neuron morphology data from the [Allen Cell Types Database](https://celltypes.brain-map.org/). The [AllenSDK](https://allensdk.readthedocs.io/en/latest/) can be used for data download.

In [ ]:
ALLEN_DIR = "allen_cell_types_database"

In [ ]:
ctc = CellTypesCache(manifest_file=f"{ALLEN_DIR}/manifest.json")

In [ ]:
MAX_CELLS = 1
SPECIES = CellTypesApi.MOUSE

In [ ]:
nm_allen_identifiers = [cell["id"] for cell in ctc.get_cells(species=[SPECIES], require_reconstruction = True)][:MAX_CELLS]
print(f"Selected a mouse neuron with identifier: {nm_allen_identifiers}")

Selected a mouse neuron with identifier: [485909730]


In [ ]:
with open(f"{ALLEN_DIR}/cells.json") as f:
    allen_cell_types_metadata = json.load(f)

In [ ]:
nm_allen_metadata = [neuron for neuron in allen_cell_types_metadata if neuron["specimen__id"] in nm_allen_identifiers]

In [ ]:
print(f"Metadata of the neuron {nm_allen_identifiers}:")
nm_allen_metadata

Metadata of the neuron [485909730]:


[{'cell_reporter_status': 'positive',
  'csl__normalized_depth': 0.478343598387418,
  'csl__x': 8881.0,
  'csl__y': 953.839501299405,
  'csl__z': 7768.22695782726,
  'donor__age': '',
  'donor__disease_state': '',
  'donor__id': 485250100,
  'donor__name': 'Cux2-CreERT2;Ai14-205530',
  'donor__race': '',
  'donor__sex': '',
  'donor__species': 'Mus musculus',
  'donor__years_of_seizure_history': '',
  'ef__adaptation': 0.0323396179505003,
  'ef__avg_firing_rate': 17.8906878969496,
  'ef__avg_isi': 55.895,
  'ef__f_i_curve_slope': 0.25,
  'ef__fast_trough_v_long_square': -49.0000038146973,
  'ef__peak_t_ramp': 2.85131166666667,
  'ef__ri': 213.124960660934,
  'ef__tau': 20.5677674593068,
  'ef__threshold_i_long_square': 70.0,
  'ef__upstroke_downstroke_ratio_long_square': 3.04293347960074,
  'ef__vrest': -76.9283905029297,
  'ephys_inst_thresh_thumb_path': '/api/v2/well_known_file_download/491381130',
  'ephys_thumb_path': '/api/v2/well_known_file_download/485911650',
  'erwkf__id': 491

### Download one mouse neuron morphology reconstructed from the selected neuron

We will be downloading one mouse neuron morphology from the [Allen Cell Types Database](https://celltypes.brain-map.org/) using the [AllenSDK](https://allensdk.readthedocs.io/en/latest/).

In [ ]:
for identifier in nm_allen_identifiers:
    ctc.get_reconstruction(identifier)

2022-07-12 09:07:05,983 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/500961530


### Download one mouse neuron electrophysiology recording from the selected neuron

We will be downloading one mouse neuron electrophysiology from the [Allen Cell Types Database](https://celltypes.brain-map.org/) using the [AllenSDK](https://allensdk.readthedocs.io/en/latest/).

In [ ]:
for identifier in nm_allen_identifiers:
    ctc.get_ephys_data(identifier)

2022-07-12 09:07:17,093 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491316386


## Transform Allen Cell Types Database Metadata to [Neuroshapes' MINDS](https://bbp-nexus.epfl.ch/datamodels/class-schemadataset.html) metadata

### Map the Allen Cell Types Database neuron morphologies metadata to Neuroshapes

In [ ]:
allen_nm_mapping = DictionaryMapping.load("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/mappings/allen_morphology_dataset.hjson")
nm_allen_resources = forge.map(nm_allen_metadata, allen_nm_mapping, na='')

### Map the Allen Cell Types Database neuron electrophysiology recording to Neuroshapes

In [ ]:
allen_ephys_mapping = DictionaryMapping.load("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/mappings/allen_ephys_dataset.hjson")
nephys_allen_resources = forge.map(nm_allen_metadata, allen_ephys_mapping, na='')

## Register

If the registration fails, try refreshing the access token and reinitializing the forge client in the _'Configure a forge client to store, manage and access datasets'_ section.

### Register the Allen Cell Types Database neuron morphology

In [ ]:
nm_allen_resources.id = forge.format("identifier", "neuronmorphologies", str(uuid.uuid4()))

In [ ]:
forge.register(nm_allen_resources)

<action> _register_one
<succeeded> True


### Register the Allen Cell Types Database neuron electrophysiology recording

In [ ]:
nephys_allen_resources.id = forge.format("identifier", "traces", str(uuid.uuid4()))

In [ ]:
forge.register(nephys_allen_resources)

<action> _register_one
<succeeded> True


## Access

### Set filters

In [ ]:
_type = "NeuronMorphology"

filters = {"type": _type}

### Run Query

In [ ]:
number_of_results = 10  # You can limit the number of results, pass `None` to fetch all the results

data = forge.search(filters, limit=number_of_results)

print(f"{str(len(data))} dataset(s) of type {_type} found")

1 dataset(s) of type NeuronMorphology found


### Display the results as pandas dataframe

In [ ]:
property_to_display = ["id","name","subject","brainLocation.brainRegion.id","brainLocation.brainRegion.label","brainLocation.layer.id","brainLocation.layer.label", "contribution","brainLocation.layer.id","brainLocation.layer.label","distribution.name","distribution.contentUrl","distribution.encodingFormat"]
reshaped_data = forge.reshape(data, keep=property_to_display)

forge.as_dataframe(reshaped_data)

,id,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.layer,contribution.type,contribution.agent.id,contribution.agent.type,contribution.agent.label,distribution.contentUrl,distribution.encodingFormat,distribution.name,name,subject.type,subject.age.period,subject.identifier,subject.name,subject.species.label,subject.strain.label
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISp5,5,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,reconstruction.swc,Cux2-CreERT2;Ai14-205530.03.02.01,Subject,Post-natal,485250100,Cux2-CreERT2;Ai14-205530,Mus musculus,Cux2-CreERT2


### Download

In [ ]:
dirpath = "./downloaded/"
forge.download(data, "distribution.contentUrl", dirpath, overwrite=True)

In [ ]:
ls ./downloaded/

reconstruction.swc


### Display a result as 3D Neuron Morphology

In [ ]:
from neurom import load_morphology
from neurom.view.plotly_impl import plot_morph3d
import IPython

In [ ]:
neuron = load_morphology(f"{dirpath}/{data[0].distribution.name}")
plot_morph3d(neuron, inline=False)
IPython.display.HTML(filename='./morphology-3D.html')

Output hidden; open in https://colab.research.google.com to view.

## Version the dataset
Tagging a dataset is equivalent to `git tag`. It allows to version a dataset.

In [ ]:
forge.tag(data, value="releaseV112")

<count> 1
<action> _tag_many
<succeeded> True


In [ ]:
# The version argument can be specified to retrieve the dataset at a given tag.

tagged_data = forge.retrieve(id=data[0].id, version="releaseV112")

In [ ]:
forge.as_dataframe(tagged_data)

,id,type,_schemaProject,apicalDendrite,brainLocation.type,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.coordinatesInBrainAtlas.valueX,brainLocation.coordinatesInBrainAtlas.valueY,brainLocation.coordinatesInBrainAtlas.valueZ,...,objectOfStudy.id,objectOfStudy.type,objectOfStudy.label,subject.type,subject.age.period,subject.identifier,subject.name,subject.species.label,subject.strain.label,tag__apical
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,"[Dataset, NeuronMorphology]",https://sandbox.bluebrainnexus.io/v1/projects/...,spiny,BrainLocation,http://api.brain-map.org/api/v2/data/Structure...,VISp5,8881.0,953.839501,7768.226958,...,http://bbp.epfl.ch/neurosciencegraph/taxonomie...,ObjectOfStudy,Single Cell,Subject,Post-natal,485250100,Cux2-CreERT2;Ai14-205530,Mus musculus,Cux2-CreERT2,intact


In [ ]:
data[0].description="Neuron Morphology from Allen"

In [ ]:
forge.update(data[0])

<action> _update_one
<succeeded> True


In [ ]:
non_tagged_data = forge.retrieve(id=data[0].id)

In [ ]:
forge.as_dataframe(non_tagged_data)

,id,type,_schemaProject,apicalDendrite,brainLocation.type,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.coordinatesInBrainAtlas.valueX,brainLocation.coordinatesInBrainAtlas.valueY,brainLocation.coordinatesInBrainAtlas.valueZ,...,objectOfStudy.id,objectOfStudy.type,objectOfStudy.label,subject.type,subject.age.period,subject.identifier,subject.name,subject.species.label,subject.strain.label,tag__apical
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,"[Dataset, NeuronMorphology]",https://sandbox.bluebrainnexus.io/v1/projects/...,spiny,BrainLocation,http://api.brain-map.org/api/v2/data/Structure...,VISp5,8881.0,953.839501,7768.226958,...,http://bbp.epfl.ch/neurosciencegraph/taxonomie...,ObjectOfStudy,Single Cell,Subject,Post-natal,485250100,Cux2-CreERT2;Ai14-205530,Mus musculus,Cux2-CreERT2,intact
